In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import tensorflow as tf
import os
import PIL
import pathlib

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [42]:
train_df = pd.read_csv('./dataset/train.csv')
train_df['Id'] = './dataset/train/' + train_df['Id'] + '.jpg'
img_names = train_df.pop('Id').to_numpy()
targets = train_df.pop('Pawpularity').to_numpy()

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,0,1,1,1,0,0,1,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0


In [43]:
image_size = 200
batch_size = 32

def parse_image_file(inputs, label):
    image = tf.io.read_file(inputs[0])
    image = tf.io.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [image_size, image_size])
    return ( (image, inputs[1]), label)

train_dataset = tf.data.Dataset.from_tensor_slices(( (img_names, train_df.to_numpy(dtype='int8')), targets ))
train_dataset = train_dataset.map(parse_image_file).shuffle(100).batch(batch_size)

In [45]:
# functional API

# inputs
image_input = tf.keras.Input(shape=(image_size, image_size, 3), name='image')
tabular_input = tf.keras.Input(shape=(train_df.shape[1],), name='tabular')

# image processing
x = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', name='cb1_conv')(image_input)
x = tf.keras.layers.MaxPooling2D(2, 2, name='cb1_pool')(x)
x = tf.keras.layers.Dropout(0.2, name='cb1_drop')(x)

x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', name='cb2_conv')(x)
x = tf.keras.layers.MaxPooling2D(2, 2, name='cb2_pool')(x)
x = tf.keras.layers.Dropout(0.2, name='cb2_drop')(x)

flat_img = tf.keras.layers.Flatten(name='flat')(x)

# tabular metadata processing
processed_tabular = tf.keras.layers.Dense(32, activation='relu', name='processed_tabular')(tabular_input)

# combining
y = tf.keras.layers.Concatenate(name='concat')([flat_img, processed_tabular])
y = tf.keras.layers.Dropout(0.3)(y)
y = tf.keras.layers.Dense(64, activation='relu')(y)
y = tf.keras.layers.Dropout(0.3)(y)
y = tf.keras.layers.Dense(16, activation='relu')(y)
y = tf.keras.layers.Dense(1)(y)
output = tf.keras.layers.Rescaling(100)(y)

model = tf.keras.Model(
    inputs = [image_input, tabular_input],
    outputs=output
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
cb1_conv (Conv2D)               (None, 200, 200, 32) 896         image[0][0]                      
__________________________________________________________________________________________________
cb1_pool (MaxPooling2D)         (None, 100, 100, 32) 0           cb1_conv[0][0]                   
__________________________________________________________________________________________________
cb1_drop (Dropout)              (None, 100, 100, 32) 0           cb1_pool[0][0]                   
______________________________________________________________________________________________

In [47]:
model.compile(
    optimizer = 'adam',
    loss = 'mean_squared_error'
)

history = model.fit(
    train_dataset,
    epochs=2
)

Epoch 1/2
310/310 [==============================] - 390s 1s/step - loss: 540.0953
Epoch 2/2
310/310 [==============================] - 415s 1s/step - loss: 438.3320
